In [94]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

In [163]:
df = pd.read_csv('gurgaon_properties_post_feature_selection_v2.csv')

In [66]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3,2,2,New Property,850.0,0,0,0,Low,Low Floor
1,flat,sector 89,0.95,2,2,2,New Property,1226.0,1,0,0,Low,Mid Floor
2,flat,sohna road,0.32,2,2,1,New Property,1000.0,0,0,0,Low,High Floor
3,flat,sector 92,1.60,3,4,3+,Relatively New,1615.0,1,0,1,High,Mid Floor
4,flat,sector 102,0.48,2,2,1,Relatively New,582.0,0,1,0,High,Mid Floor


In [67]:
df.shape

(3553, 13)

In [68]:
df['furnishing_type'].value_counts()

furnishing_type
0    2349
1    1017
2     187
Name: count, dtype: int64

In [69]:
# 0 -> unfurnished
# 1 -> semifurnished
# 2 -> furnished
df['furnishing_type'] = df['furnishing_type'].replace({0.0:'unfurnished',1.0:'semifurnished',2.0:'furnished'})

In [70]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3,2,2,New Property,850.0,0,0,unfurnished,Low,Low Floor
1,flat,sector 89,0.95,2,2,2,New Property,1226.0,1,0,unfurnished,Low,Mid Floor
2,flat,sohna road,0.32,2,2,1,New Property,1000.0,0,0,unfurnished,Low,High Floor
3,flat,sector 92,1.60,3,4,3+,Relatively New,1615.0,1,0,semifurnished,High,Mid Floor
4,flat,sector 102,0.48,2,2,1,Relatively New,582.0,0,1,unfurnished,High,Mid Floor


In [71]:
X = df.drop(columns=['price'])
y = df['price']

In [72]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

### Ordinal Encoding

In [73]:
X.columns

Index(['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category'],
      dtype='object')

In [74]:
columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [75]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [76]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [77]:
pipeline

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [78]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [79]:
scores.mean(),scores.std()

(np.float64(0.7368361616611377), np.float64(0.025950760796473977))

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [81]:
pipeline.fit(X_train,y_train)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [82]:
y_pred = pipeline.predict(X_test)

In [83]:
y_pred = np.expm1(y_pred)

In [84]:
mean_absolute_error(np.expm1(y_test),y_pred)

0.9528101998211803

In [85]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [95]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [96]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [97]:
model_output

[['linear_reg', np.float64(0.7368361616611377), 0.9528101998211803],
 ['svr', np.float64(0.7641639653587954), 0.896732243853289],
 ['ridge', np.float64(0.736839673146599), 0.9528811530950675],
 ['LASSO', np.float64(0.0627965614963962), 1.6500145253841618],
 ['decision tree', np.float64(0.775240954871382), 0.7294142278599473],
 ['random forest', np.float64(0.8820182836508559), 0.5583420147879354],
 ['extra trees', np.float64(0.8710375439958055), 0.5863771455540546],
 ['gradient boosting', np.float64(0.8747588067725713), 0.5950839528421817],
 ['adaboost', np.float64(0.7570805858695715), 0.8846644334548662],
 ['mlp', np.float64(0.8074686794998767), 0.7093325787198225],
 ['xgboost', np.float64(0.8942665583291758), 0.5281748134425923]]

In [98]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [99]:
model_df.sort_values(['mae'])

,name,r2,mae
10,xgboost,0.894267,0.528175
5,random forest,0.882018,0.558342
6,extra trees,0.871038,0.586377
7,gradient boosting,0.874759,0.595084
9,mlp,0.807469,0.709333
4,decision tree,0.775241,0.729414
8,adaboost,0.757081,0.884664
1,svr,0.764164,0.896732
0,linear_reg,0.736836,0.952810
2,ridge,0.736840,0.952881


### OneHotEncoding

In [100]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first'),['sector','agePossession','furnishing_type'])
    ], 
    remainder='passthrough'
)

In [101]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [102]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [103]:
scores.mean()

np.float64(0.8546035043590976)

In [104]:
scores.std()

np.float64(0.014727402806347703)

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [106]:
pipeline.fit(X_train,y_train)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [107]:
y_pred = pipeline.predict(X_test)

In [108]:
y_pred = np.expm1(y_pred)

In [109]:
mean_absolute_error(np.expm1(y_test),y_pred)

0.6805035408121454

In [110]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [111]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [112]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [113]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [114]:
model_df.sort_values(['mae'])

,name,r2,mae
10,xgboost,0.895029,0.481188
6,extra trees,0.898977,0.485863
5,random forest,0.893574,0.516813
9,mlp,0.867547,0.588281
7,gradient boosting,0.876424,0.603135
4,decision tree,0.803202,0.647202
0,linear_reg,0.854604,0.680504
2,ridge,0.854830,0.684695
1,svr,0.769618,0.888280
8,adaboost,0.758690,0.897777


### OneHotEncoding With PCA

In [115]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['sector','agePossession'])
    ], 
    remainder='passthrough'
)

In [116]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=0.95)),
    ('regressor', LinearRegression())
])

In [117]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [118]:
scores.mean()

np.float64(0.066043690599685)

In [119]:
scores.std()

np.float64(0.014281363889817449)

In [120]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components=0.95)),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [121]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [122]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [123]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [124]:
model_df.sort_values(['mae'])

,name,r2,mae
5,random forest,0.761128,0.725709
6,extra trees,0.742618,0.738888
4,decision tree,0.691372,0.806335
10,xgboost,0.615052,0.965773
7,gradient boosting,0.608441,1.076768
8,adaboost,0.300847,1.439891
1,svr,0.217229,1.465816
9,mlp,0.215164,1.514900
2,ridge,0.066044,1.648242
0,linear_reg,0.066044,1.648242


### Target Encoder

In [129]:
import category_encoders as ce

columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
)

In [ ]:
# !pip install category_encoders

   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.6 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.6 MB 894.9 kB/s eta 0:00:11
   -- ------------------------------------- 0.5/9.6 MB 894.9 kB/s eta 0:00:11
   --- ------------------------------------ 0.8/9.6 MB 855.0 kB/s eta 0:00:11
   --- ------------------------------------ 0.8/9.6 MB 855.0 kB/s eta 0:00:11
   ----- ---------------------------------- 1.3/9.6 MB 863.0 kB/s eta 0:00:10
   ------ --------------------------------- 1.6/9.6 MB 973.9 kB/s eta 0:00:09
   ------- -------------------------------- 1.8/9.6 MB 951.5 kB/s eta 0:00:09
   ------- -------------------------------- 1.8/9.6 MB 951.5 kB/s eta 0:00:09
   ------- -------------------------------- 1.8/9.6 MB 951.5 kB/s eta 0:00:09
   --------- ---------

In [130]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [131]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [132]:
scores.mean(),scores.std()

(np.float64(0.8297944090523515), np.float64(0.0130849330015545))

In [133]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [134]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [135]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [136]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [137]:
model_df.sort_values(['mae'])

,name,r2,mae
10,xgboost,0.903796,0.499207
5,random forest,0.904797,0.499790
6,extra trees,0.902586,0.500649
7,gradient boosting,0.890782,0.533533
4,decision tree,0.827307,0.564178
9,mlp,0.854756,0.641944
0,linear_reg,0.829794,0.761000
2,ridge,0.829816,0.761572
8,adaboost,0.817465,0.764672
1,svr,0.783108,0.869947


### Hyperparameter Tuning

In [138]:
from sklearn.model_selection import GridSearchCV

In [139]:
param_grid = {
    'regressor__n_estimators': [50, 100, 200, 300],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__max_samples':[0.1, 0.25, 0.5, 1.0],
    'regressor__max_features': ['auto', 'sqrt']
}

In [140]:
columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
)

In [141]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

In [142]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [143]:
search = GridSearchCV(pipeline, param_grid, cv=kfold, scoring='r2', n_jobs=-1, verbose=4)

In [144]:
search.fit(X, y_transformed)

Fitting 10 folds for each of 128 candidates, totalling 1280 fits


c:\Users\wasia\Desktop\Data Science\Data science Property Price Prediction Project\env313\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
640 fits failed out of a total of 1280.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
296 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\wasia\Desktop\Data Science\Data science Property Price Prediction Project\env313\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\wasia\Desktop\Data Science\Data science Property Price Prediction Project\env313\Lib\site-packages\sklearn\ba

,estimator,Pipeline(step...Regressor())])
,param_grid,"{'regressor__max_depth': [None, 10, ...], 'regressor__max_features': ['auto', 'sqrt'], 'regressor__max_samples': [0.1, 0.25, ...], 'regressor__n_estimators': [50, 100, ...]}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,4
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...), ...]"


In [145]:
final_pipe = search.best_estimator_

In [146]:
search.best_params_

{'regressor__max_depth': 30,
 'regressor__max_features': 'sqrt',
 'regressor__max_samples': 1.0,
 'regressor__n_estimators': 100}

In [147]:
search.best_score_

np.float64(0.9024787562841127)

In [148]:
final_pipe.fit(X,y_transformed)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


### Exporting the model

In [149]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['sector','agePossession'])
    ], 
    remainder='passthrough'
)

In [150]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=500))
])

In [151]:
pipeline.fit(X,y_transformed)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [152]:
import pickle

with open('pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [153]:
with open('df.pkl', 'wb') as file:
    pickle.dump(X, file)

In [154]:
X

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,3,2,2,New Property,850.0,0,0,unfurnished,Low,Low Floor
1,flat,sector 89,2,2,2,New Property,1226.0,1,0,unfurnished,Low,Mid Floor
2,flat,sohna road,2,2,1,New Property,1000.0,0,0,unfurnished,Low,High Floor
3,flat,sector 92,3,4,3+,Relatively New,1615.0,1,0,semifurnished,High,Mid Floor
4,flat,sector 102,2,2,1,Relatively New,582.0,0,1,unfurnished,High,Mid Floor
...,...,...,...,...,...,...,...,...,...,...,...,...
3548,flat,sector 84,2,2,1,Relatively New,532.0,0,0,unfurnished,Medium,Mid Floor
3549,house,sector 109,5,5,3+,Relatively New,6228.0,1,1,unfurnished,High,Low Floor
3550,flat,sector 2,1,1,1,Moderately Old,665.0,0,0,semifurnished,Medium,Mid Floor
3551,house,sector 43,5,6,3,Moderately Old,5490.0,1,1,unfurnished,Medium,Mid Floor


### Trying out the predictions

In [155]:
X.columns

Index(['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category'],
      dtype='object')

In [156]:
X.iloc[0].values

array(['flat', 'sector 36', np.int64(3), np.int64(2), '2', 'New Property',
       np.float64(850.0), np.int64(0), np.int64(0), 'unfurnished', 'Low',
       'Low Floor'], dtype=object)

In [157]:
data = [['house', 'sector 102', 4, 3, '3+', 'New Property', 2750, 0, 0, 'unfurnished', 'Low', 'Low Floor']]
columns = ['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category']

# Convert to DataFrame
one_df = pd.DataFrame(data, columns=columns)

one_df


,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,house,sector 102,4,3,3+,New Property,2750,0,0,unfurnished,Low,Low Floor


In [158]:
np.expm1(pipeline.predict(one_df))

array([3.08412252])

In [159]:
X.dtypes

property_type       object
sector              object
bedRoom              int64
bathroom             int64
balcony             object
agePossession       object
built_up_area      float64
servant room         int64
store room           int64
furnishing_type     object
luxury_category     object
floor_category      object
dtype: object

In [160]:
sorted(X['sector'].unique().tolist())

['dwarka expressway',
 'gwal pahari',
 'manesar',
 'sector 1',
 'sector 10',
 'sector 102',
 'sector 103',
 'sector 104',
 'sector 105',
 'sector 106',
 'sector 107',
 'sector 108',
 'sector 109',
 'sector 11',
 'sector 110',
 'sector 111',
 'sector 112',
 'sector 113',
 'sector 12',
 'sector 13',
 'sector 14',
 'sector 15',
 'sector 17',
 'sector 2',
 'sector 21',
 'sector 22',
 'sector 23',
 'sector 24',
 'sector 25',
 'sector 26',
 'sector 27',
 'sector 28',
 'sector 3',
 'sector 30',
 'sector 31',
 'sector 33',
 'sector 36',
 'sector 37',
 'sector 37d',
 'sector 38',
 'sector 39',
 'sector 4',
 'sector 40',
 'sector 41',
 'sector 43',
 'sector 45',
 'sector 46',
 'sector 47',
 'sector 48',
 'sector 49',
 'sector 5',
 'sector 50',
 'sector 51',
 'sector 52',
 'sector 53',
 'sector 54',
 'sector 55',
 'sector 56',
 'sector 57',
 'sector 58',
 'sector 59',
 'sector 6',
 'sector 60',
 'sector 61',
 'sector 62',
 'sector 63',
 'sector 63a',
 'sector 65',
 'sector 66',
 'sector 67',
 'se

In [162]:
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pymysql
import pyodbc

# Load .env variables
from dotenv import load_dotenv
print(load_dotenv())

# Create MySQL connection (AWS RDS)
mysql_engine = create_engine(
    f"mysql+pymysql://{os.getenv('MYSQL_USER')}:{os.getenv('MYSQL_PASS')}@{os.getenv('MYSQL_HOST')}/{os.getenv('MYSQL_DB')}"
)

# Create MSSQL connection
mssql_engine = create_engine(
    f"mssql+pyodbc://{os.getenv('MSSQL_USER')}:{os.getenv('MSSQL_PASS')}@{os.getenv('MSSQL_HOST')}/{os.getenv('MSSQL_DB')}?driver={os.getenv('MSSQL_DRIVER')}&Encrypt=yes&TrustServerCertificate=no"
)

True


In [164]:
df.shape

(3553, 13)

In [165]:
df.to_sql("gurgaon_properties_post_feature_selection_v2", con=mysql_engine, if_exists='replace', index=False)

3553

In [167]:
df.to_sql("gurgaon_properties_post_feature_selection_v2", con=mssql_engine, if_exists='replace', index=False)

11